# Import Libraries

In [ ]:
import torch
import torch.nn.functional as F
!pip install facenet-pytorch
!pip install gradio==3.50
from facenet_pytorch import MTCNN, InceptionResnetV1
!pip install facenet_pytorch
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

import warnings
warnings.filterwarnings("ignore")
import numpy as np
from PIL import Image
import cv2
import gradio as gr
!pip install captum
from captum.attr import IntegratedGradients
from captum.attr import visualization as viz
from captum.attr import InterpretableInput
import tqdm as tqdm

# Download and Load Model

In [ ]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()

In [ ]:
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

#checkpoint = torch.load("epoch28deepfake_detection.pt", map_location=torch.device('cpu'))
#model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

# Model Inference 

In [ ]:
def predict(input_image:Image.Image):
    """Predict the label of the input_image"""
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')
    
    face = face.unsqueeze(0) # add the batch dimension
    
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)
    
    # convert the face into a numpy array to be able to plot it
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
    prev_face = prev_face.astype('uint8')

    face = face.to(DEVICE)
    face = face.to(torch.float32)
    face = face / 255.0
    print(face.size())
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()


    output = 1
    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        prediction = "real" if output.item() < 0.5 else "fake"
        
        real_prediction = 1 - output.item()
        fake_prediction = output.item()
        
        confidences = {
            'real': real_prediction,
            'fake': fake_prediction
        }
    ig = IntegratedGradients(model)
    attributions = ig.attribute(face, n_steps=200 , target = 0  )
    attributions = attributions.squeeze().permute(1, 2, 0).cpu().detach().numpy()
    face = face.squeeze().permute(1, 2, 0).cpu().detach().numpy()
    fig = viz.visualize_image_attr(attributions , face , 'original_image')
    fig = viz.visualize_image_attr(attributions , face , 'alpha_scaling')
    
    
    return confidences


# Gradio Interface

In [ ]:

interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Image(label="Input Image", type="pil")
    ],
    outputs=[
        gr.Label(label="Class"),
      
    ],
).launch(share = True)


In [ ]:
#from torchvision.io import read_image 
#print(predict(Image.open("/kaggle/input/deepfake-and-real-images/Dataset/Train/Fake/fake_10000.jpg")))

In [ ]:
def pred_from_model(threshold , model_output):
    if(model_output > threshold):
        return 1
    return 0
    

In [ ]:
def accuracy(y_pred, y_true):
    correct = (y_pred == y_true).sum().item()
    total = y_true.size(0)
    return correct / total

In [ ]:
def precision(y_pred, y_true):
    true_positives = ((y_pred == 1) & (y_true == 1)).sum().item()
    false_positives = ((y_pred == 1) & (y_true == 0)).sum().item()
    return true_positives / (true_positives + false_positives + 1e-8)


In [ ]:
def recall(y_pred, y_true):
    true_positives = ((y_pred == 1) & (y_true == 1)).sum().item()
    false_negatives = ((y_pred == 0) & (y_true == 1)).sum().item()
    return true_positives / (true_positives + false_negatives + 1e-8)



In [ ]:
def f1_score(y_pred, y_true):
    prec = precision(y_pred, y_true)
    rec = recall(y_pred, y_true)
    return 2 * (prec * rec) / (prec + rec + 1e-8)
